In [16]:
import pandas as pd
df = pd.read_csv('differences_zbmath_data_dump20231221_TO_zbmath_data_dump20240912.csv', sep='\t', nrows=10)

df
#for each col that doesnt contian new, compare with its new equivalent if the new is not nan
row = df.iloc[0]
print(row["de_number"])
if not row["doi_new"].isna():
    #maybe move this stuff lower to where it's acutally determined if we need something new
    document_title = get_info_from_doi(
            doi=info_dict["doi"].strip(), key="document_title"
        )
    journal_string = get_info_from_doi(
                            doi=info_dict["doi"].strip(), key="journal"
                        )
for col in list(df):
    if "_new" not in col and col != "de_number":
        if not pd.isna(row[col+"_new"]):
            old_val = row[col]
            new_val = row[col+"_new"]
            print(col)
            print(old_val)
            print(new_val)

#maybe strip each value once
#those that only have one value: doi, document_title, zbl_id
#those that don't have only one, but I am still taking the last one: journal
#those that have several semicolon values: serial, author_ids

1566037


AttributeError: 'numpy.float64' object has no attribute 'isna'

In [14]:
list(df)

['de_number',
 'creation_date',
 'author',
 'author_ids',
 'document_title',
 'source',
 'classifications',
 'language',
 'links',
 'keywords',
 'doi',
 'publication_year',
 'serial',
 'zbl_id',
 'review_text',
 'review_sign',
 'reviewer_id',
 'creation_date_new',
 'author_new',
 'author_ids_new',
 'document_title_new',
 'source_new',
 'classifications_new',
 'language_new',
 'links_new',
 'keywords_new',
 'doi_new',
 'publication_year_new',
 'serial_new',
 'zbl_id_new',
 'review_text_new',
 'review_sign_new',
 'reviewer_id_new']

In [4]:
compare_large_dfs(
        data_file_1="../../../../zbmath_data_dump20231221-160126.csv",
        data_file_2='../../../../zbmath_data_dump20240912-142641.csv',
        hash_file_1='../../../../zbmath_data_dump20231221-160126_hash.csv',
        hash_file_2="../../../../zbmath_data_dump20240912-142641_hash.csv",
        de_col="de_number",
        chunksize=50_000,
        output_file="differences.csv"
    )

Reading hash files into memory...
Found 4519844 de_numbers with different hashes.


/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (7,8,9,12,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (7,8,9,12,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk2 in df2_iter:
/tmp/ipykernel_29963/3859913072.py:75: DtypeWarning: Columns (7,8,9,12,14,15,16,17) have mixed types. Specify dtype option on import o

Done! Differences written to 'differences.csv'.


In [9]:
old_hash = pd.read_csv('../../../../zbmath_data_dump20231221-160126_hash.csv', sep='\t')
new_hash = pd.read_csv('../../../../zbmath_data_dump20240912-142641_hash.csv', sep='\t')

In [12]:
old_hash.columns

Index(['de_number', 'hash'], dtype='object')

In [5]:
old_ones_not_in_new = old_de - new_de 
new_ones_not_in_old = new_de - old_de

In [5]:
test = pd.read_csv("differences.csv", sep="\t", nrows=200)

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
test

,creation_date,author,author_ids,document_title,source,classifications,language,links,keywords,doi,publication_year,serial,zbl_id,review_text,review_sign,reviewer_id,creation_date_new,author_new,author_ids_new,document_title_new,source_new,classifications_new,language_new,links_new,keywords_new,doi_new,publication_year_new,serial_new,zbl_id_new,review_text_new,review_sign_new,reviewer_id_new
0,2002-01-02T23:00:00Z,"Önder, Turgut",onder.turgut,Equivariant cross sections of complex Stiefel ...,"Topology Appl. 109, No. 1, 107-125 (2001).",57R91;55R91;55R50;57S25;57R25;55R25;22A25,English,NaN,complex Stiefel manifolds;equivariant fibre sp...,10.1016/S0166-8641(99)00149-2,2001,"Elsevier (North-Holland), Amsterdam;Topology a...",976.57032,"Let \(G\) be a topological group, \(M\) be a u...",Vladimir L. Popov (Moskva),popov.vladimir-leonidovich,0001-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Topology and its Applications,NaN,NaN,Vladimir L. Popov,NaN
1,2001-07-23T22:00:00Z,"Luukkainen, Jouni",luukkainen.jouni,Lipschitz and quasiconformal approximation of ...,"Topology Appl. 109, No. 1, 1-40 (2001).",57N35;30C65;57R12,English,NaN,quasisymmetric;locally flat;diffeomorphism,10.1016/S0166-8641(99)00145-5,2001,"Elsevier (North-Holland), Amsterdam;Topology a...",964.57023,This paper deals with manifolds in the categor...,Jouni Luukkainen (Helsinki),luukkainen.jouni,0001-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Topology and its Applications,NaN,This paper deals with manifolds in the categor...,Jouni Luukkainen,NaN
2,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,alexandru.victor;gosoniu.marin,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,11-01,zbMATH Open Web Interface contents unavailable...,NaN,NaN,NaN,1998,zbMATH Open Web Interface contents unavailable...,1079.11500,zbMATH Open Web Interface contents unavailable...,NaN,NaN,0001-01-01T00:00:00Z,"Alexandru, Victor;Goşoniu, Marin",NaN,Elements of number theory,Bucuresti: Editura Universităţii Bucureşti (IS...,NaN,Romanian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,arsinte.vasile,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,26-01,zbMATH Open Web Interface contents unavailable...,NaN,NaN,NaN,1998,zbMATH Open Web Interface contents unavailable...,1079.26500,zbMATH Open Web Interface contents unavailable...,NaN,NaN,0001-01-01T00:00:00Z,"Arsinte, Vasile",NaN,Elementary problems of integral calculus,Bucureşti: Editura Universităţii Bucureşti (IS...,NaN,Romanian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,badescu.lucian,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,51-01;53-01,zbMATH Open Web Interface contents unavailable...,NaN,NaN,NaN,1999,zbMATH Open Web Interface contents unavailable...,1130.51300,zbMATH Open Web Interface contents unavailable...,NaN,NaN,0001-01-01T00:00:00Z,"Bădescu, Lucian",NaN,Lectures on geometry,Bucureşti: Editura Universităţii din Bucureşti...,NaN,Romanian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,newborn.monty,zbMATH Open Web Interface contents unavailable...,zbMATH Open Web Interface contents unavailable...,68T15;68-01,zbMATH Open Web Interface contents unavailable...,NaN,predicate calculus;inference procedures;semant...,NaN,2001,zbMATH Open Web Interface contents unavailable...,963.68176,zbMATH Open Web Interface contents unavailable...,Doina Tătar (Cluj),tatar.doina,0001-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doina Tătar,NaN
196,2001-02-19T23:00:00Z,zbMATH Open Web Interface contents unavailable...,fru

In [8]:
test.shape

(200, 32)

In [7]:
len(new_ones_not_in_old)

319375

In [8]:
import csv
import pandas as pd

def read_de_numbers_csv(filename, de_column_name="de_number"):
    de_values = set()
    with open(filename, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter="\t")
        for row in reader:
            de_values.add(row[de_column_name])
    return de_values

def subset_large_file_by_ids(
    input_file, 
    output_file, 
    de_values, 
    id_column="de_number", 
    chunksize=100000
):
    """
    Reads input_file in chunks, filters rows where id_column is in de_values,
    and writes them to output_file without loading the entire file into memory.
    """
    # You can tune chunksize depending on your memory constraints.
    first_chunk = True
    with pd.read_csv(input_file, sep='\t', chunksize=chunksize) as reader:
        for chunk in reader:
            # Filter this chunk
            chunk_filtered = chunk[chunk[id_column].isin(de_values)]
            
            # Write out the filtered rows
            chunk_filtered.to_csv(
                output_file,
                sep='\t',
                index=False,
                header=first_chunk,  # write header only for the first chunk
                mode='a'             # append to the file
            )
            first_chunk = False

old_file = '../../../../zbmath_data_dump20231221-160126_hash.csv'
new_file = '../../../../zbmath_data_dump20240912-142641_hash.csv'
new_data_file = '../../../../zbmath_data_dump20240912-142641.csv'
old_name = old_file.split('/')[-1].split('-')[0]
new_name = new_file.split('/')[-1].split('-')[0]
old_de = read_de_numbers_csv(old_file)
new_de = read_de_numbers_csv(new_file)

old_ones_not_in_new = [int(x) for x in list(old_de - new_de)]
new_ones_not_in_old = [int(x) for x in list(new_de - old_de)]
with open(f'deleted_{old_name}_TO_{new_name}.tsv', 'a+') as f:
    f.write("de_number\n")
    for de in old_ones_not_in_new:
        f.write(str(de)+'\n')
subset_large_file_by_ids(
    input_file=new_data_file,
    output_file=f'new_{old_name}_TO_{new_name}.tsv',
    de_values=new_ones_not_in_old,
    id_column="de_number",
    chunksize=100000
)

/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (7,8,9,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


In [9]:
len(old_ones_not_in_new)

2449

In [10]:
len(new_ones_not_in_old)

319375

In [11]:
type(list(new_ones_not_in_old)[1])

int